In [1]:
# 在 Notebook 裡跑
import sys
!{sys.executable} -m pip install -U ipywidgets jupyterlab_widgets widgetsnbextension

In [2]:
import ipywidgets, jupyterlab_widgets, widgetsnbextension
print("ipywidgets:", ipywidgets.__version__)
print("jupyterlab_widgets:", jupyterlab_widgets.__version__)
print("widgetsnbextension:", widgetsnbextension.__version__)

ipywidgets: 8.1.7
jupyterlab_widgets: 3.0.15
widgetsnbextension: 4.0.14


In [3]:
import ipywidgets as W
from IPython.display import display

display(W.HTML("<b>ipywidgets OK</b>"))
display(W.IntSlider(value=3, min=0, max=10, description="測試滑桿"))

HTML(value='<b>ipywidgets OK</b>')

IntSlider(value=3, description='測試滑桿', max=10)

In [55]:
# --- 麥當勞點餐機 (ipywidgets)第一代 — PNG / 圖片在中欄 / 結帳後產生收據 / 高品質顯示 ---
# pip install ipywidgets pillow

from IPython.display import display
import ipywidgets as W
from datetime import datetime
import re, os, io
from PIL import Image

# ========= 資料 =========
PRICES_MAIN = {"bigmac": 89, "mcchicken": 79, "filetofish": 95, "dblcheese": 99}
PRICES_FRIES = {"fries_s": 35, "fries_m": 45, "fries_l": 55}
PRICES_NUGGETS = {"nuggets4": 49, "nuggets6": 65, "nuggets10": 109}
PRICES_SNACK_OTHER = {"apple_pie": 42, "corn_soup": 55, "side_salad": 50, "sundae_choco": 45}
PRICES_DRINK = {"coke": 38, "sprite": 36, "iced_tea": 32, "hot_coffee": 40}

NAME_MAIN = {"bigmac": "大麥克", "mcchicken": "麥香雞", "filetofish": "麥香魚", "dblcheese": "雙層牛肉吉事堡"}
NAME_FRIES_SIZE = {"fries_s": "小薯", "fries_m": "中薯", "fries_l": "大薯"}
NAME_FRIES_SALT = {"nosalt": "無鹽", "salt": "有鹽"}
NAME_NUGGETS_SIZE = {"nuggets4": "4塊", "nuggets6": "6塊", "nuggets10": "10塊"}
NAME_NUGGETS_SAUCE = {"none": "不需要醬料", "sweet_chili": "甜辣醬", "mustard": "芥末醬", "bbq": "BBQ醬"}
NAME_SNACK_OTHER = {"apple_pie": "蘋果派", "corn_soup": "玉米濃湯", "side_salad": "田園沙拉", "sundae_choco": "巧克力聖代"}
NAME_DRINK = {"coke": "可口可樂(去冰)", "sprite": "雪碧", "iced_tea": "冰紅茶", "hot_coffee": "熱美式"}

# 圖片（PNG）
IMG_MAP = {
    "bigmac": "bigmac.png", "mcchicken": "mcchicken.png", "filetofish": "filetofish.png", "dblcheese": "dblcheese.png",
    "fries_s": "fries_s.png", "fries_m": "fries_m.png", "fries_l": "fries_l.png",
    "nuggets4": "nuggets4.png", "nuggets6": "nuggets6.png", "nuggets10": "nuggets10.png",
    "apple_pie": "apple_pie.png", "corn_soup": "corn_soup.png", "side_salad": "side_salad.png", "sundae_choco": "sundae_choco.png",
    "coke": "coke.png", "sprite": "sprite.png", "iced_tea": "iced_tea.png", "hot_coffee": "hot_coffee.png",
}
ASSETS_DIR = "./assets"
DEFAULT_IMG = "none.png"

ORDER_COUNTER = {"n": 0}
CART = {}  # sku -> {name_display, unit_price, qty, category}

# ========= 圖片：只縮不放 & 高品質 =========
def load_image_for_widget(key: str, max_w=380, max_h=300):
    """回傳 (bytes, w, h)。若超出上限則等比例縮小；小圖不放大。"""
    filename = IMG_MAP.get(key, DEFAULT_IMG)
    path = os.path.join(ASSETS_DIR, filename)
    if not os.path.exists(path):
        path = os.path.join(ASSETS_DIR, DEFAULT_IMG)
    try:
        with Image.open(path) as im:
            im = im.convert("RGBA")
            ow, oh = im.size
            scale = min(max_w / ow, max_h / oh, 1.0)  # 只縮不放
            if scale < 1.0:
                nw, nh = int(ow * scale), int(oh * scale)
                im = im.resize((nw, nh), resample=Image.LANCZOS)
            else:
                nw, nh = ow, oh
            bio = io.BytesIO()
            im.save(bio, format="PNG")
            return bio.getvalue(), nw, nh
    except Exception:
        return b"", 0, 0

def currency(n: int) -> str: return f"${n:,}"
def next_order_id() -> str:
    ORDER_COUNTER["n"] += 1
    return f"MCD-{datetime.now().strftime('%Y%m%d-%H%M%S')}-{ORDER_COUNTER['n']:03d}"
def tw_tax_id_ok(b: str) -> bool: return bool(re.fullmatch(r"\d{8}", b))
def carrier_hint_ok(c: str) -> bool: return bool(re.fullmatch(r"/[A-Z0-9.+-]{7}", c.upper()))

# ========= UI 基礎 =========
title = W.HTML("<h2>🍔 麥當勞點餐機（ipywidgets / PNG）</h2>")
tb_category = W.ToggleButtons(options=[("主餐","main"),("點心","snack"),("飲料","drink")], value="main", description="類別")
product_area = W.VBox()

# 圖片放中欄（先建空 Image，實際寬高由 set_preview 設）
img_preview = W.Image(format="png")
img_center = W.HBox([img_preview], layout=W.Layout(justify_content="center"))

# 右欄：明細/小計/備註/結帳
lbl_subtotal = W.HTML("<b>小計：$0</b>")
cart_area = W.VBox()
txt_note = W.Textarea(placeholder="請輸入備註（例如：飲料少冰、餐點不加洋蔥）", description="備註",
                      layout=W.Layout(width="100%", height="70px"))
cb_bag = W.Checkbox(value=False, description="需要購物袋 (+1)")
txt_tax_id = W.Text(value="", placeholder="8 碼數字，例如 12345678", description="統編")
txt_carrier = W.Text(value="", placeholder="/AB12345（手機條碼，可留白）", description="載具")
btn_checkout = W.Button(description="結帳", button_style="success", icon="credit-card")
out_receipt = W.Output()
checkout_panel = W.VBox([cb_bag, txt_tax_id, txt_carrier, btn_checkout, out_receipt])

# 三欄
left_col = W.VBox([tb_category])
mid_col  = W.VBox([product_area, W.HTML("<hr>"), W.HTML("<b>圖片預覽</b>"), img_center])
right_col = W.VBox([
    W.HTML("<b>購物車明細</b>"), cart_area, lbl_subtotal,
    W.HTML("<hr>"), txt_note,
    W.HTML("<hr>"), W.HTML("<b>結帳</b>"), checkout_panel
])
ui = W.HBox([left_col, mid_col, right_col],
            layout=W.Layout(width="100%", justify_content="space-between", align_items="flex-start", gap="24px"))

# ========= 購物車 =========
def compute_subtotal() -> int:
    return sum(i["unit_price"]*i["qty"] for i in CART.values()) + (1 if cb_bag.value else 0)
def update_subtotal_label(): lbl_subtotal.value = f"<b>小計：{currency(compute_subtotal())}</b>"
def remove_cart_item(sku: str):
    if sku in CART: del CART[sku]
    render_cart()
def on_qty_changed(change, sku: str):
    if change["name"]!="value": return
    try: newq = int(change["new"])
    except: newq = 0
    newq = max(0, newq)
    if newq==0: remove_cart_item(sku)
    else:
        CART[sku]["qty"]=newq
        render_cart()
def render_cart():
    rows=[]
    if not CART:
        rows.append(W.HTML("<i>購物車目前是空的</i>"))
    else:
        rows.append(W.HBox([
            W.HTML("<b>品項</b>", layout=W.Layout(width="55%")),
            W.HTML("<b>單價</b>", layout=W.Layout(width="12%")),
            W.HTML("<b>份數</b>", layout=W.Layout(width="16%")),
            W.HTML("<b>小計</b>", layout=W.Layout(width="12%")),
            W.HTML("", layout=W.Layout(width="5%")),
        ]))
        for sku,item in CART.items():
            name, unit, qty = item["name_display"], item["unit_price"], item["qty"]
            line = unit*qty
            name_w = W.HTML(name, layout=W.Layout(width="55%"))
            unit_w = W.Label(currency(unit), layout=W.Layout(width="12%"))
            qty_w  = W.IntText(value=qty, min=0, step=1, layout=W.Layout(width="90%"))
            qty_w.observe(lambda ch, s=sku: on_qty_changed(ch, s), names="value")
            sub_w  = W.Label(currency(line), layout=W.Layout(width="12%"))
            btn_del= W.Button(icon="trash", tooltip="刪除", layout=W.Layout(width="32px"))
            btn_del.on_click(lambda b, s=sku: remove_cart_item(s))
            rows.append(W.HBox([name_w, unit_w, W.HBox([qty_w], layout=W.Layout(width="16%")), sub_w, btn_del]))
    cart_area.children = rows
    update_subtotal_label()

# ========= 圖片預覽 =========
def set_preview(key: str):
    data, w, h = load_image_for_widget(key)  # 只縮不放
    img_preview.value = data
    if w and h:
        img_preview.width = w
        img_preview.height = h

# ========= 加入購物車 =========
def add_to_cart(sku: str, name_display: str, unit_price: int, qty: int, category: str):
    if qty<=0: return
    if sku in CART: CART[sku]["qty"] += qty
    else: CART[sku] = {"name_display": name_display, "unit_price": unit_price, "qty": qty, "category": category}
    render_cart()

# ========= 中欄商品卡 =========
status_msg = W.HTML("")

def build_main_card():
    dd_item = W.Dropdown(options=[(NAME_MAIN[k], k) for k in PRICES_MAIN], value="bigmac", description="主餐")
    qty     = W.IntText(value=1, min=1, step=1, description="份數")
    btn_add = W.Button(description="加入購物車", button_style="primary", icon="plus")
    status_msg.value = ""
    dd_item.observe(lambda ch: set_preview(dd_item.value), names="value")
    set_preview(dd_item.value)
    def _add(_):
        pid = dd_item.value
        add_to_cart(pid, NAME_MAIN[pid], PRICES_MAIN[pid], qty.value, "main")
        status_msg.value = f"<span style='color:green'>已加入：{NAME_MAIN[pid]} × {qty.value}</span>"
    btn_add.on_click(_add)
    return W.VBox([W.HBox([dd_item, qty, btn_add]), status_msg])

def build_snack_card():
    dd_kind = W.ToggleButtons(options=[("薯條","fries"),("麥克雞塊","nuggets"),("其他點心","other")], value="fries", description="點心類型")

    # 薯條
    dd_fries_size = W.Dropdown(options=[(NAME_FRIES_SIZE[k],k) for k in PRICES_FRIES], value="fries_m", description="尺寸")
    dd_fries_salt = W.Dropdown(options=[("無鹽","nosalt"),("有鹽","salt")], value="salt", description="口味")
    qty_fries     = W.IntText(value=1, min=1, step=1, description="份數")
    btn_add_fries = W.Button(description="加入購物車", button_style="primary", icon="plus")
    dd_fries_size.observe(lambda ch: set_preview(dd_fries_size.value), names="value")
    set_preview(dd_fries_size.value)
    def add_fries(_):
        size, salt = dd_fries_size.value, dd_fries_salt.value
        name = f"{NAME_FRIES_SIZE[size]}（{NAME_FRIES_SALT[salt]}）"
        add_to_cart(f"{size}|{salt}", name, PRICES_FRIES[size], qty_fries.value, "snack")
        status_msg.value = f"<span style='color:green'>已加入：{name} × {qty_fries.value}</span>"
    btn_add_fries.on_click(add_fries)
    fries_card = W.VBox([W.HBox([dd_fries_size, dd_fries_salt]), W.HBox([qty_fries, btn_add_fries])])

    # 麥克雞塊
    dd_n_size     = W.Dropdown(options=[(NAME_NUGGETS_SIZE[k],k) for k in PRICES_NUGGETS], value="nuggets6", description="份量")
    cb_need_sauce = W.Checkbox(value=True, description="需要醬料")
    dd_n_sauce    = W.Dropdown(options=[("不需要","none"),("甜辣醬","sweet_chili"),("芥末醬","mustard"),("BBQ醬","bbq")],
                               value="sweet_chili", description="醬料")
    qty_n         = W.IntText(value=1, min=1, step=1, description="份數")
    btn_add_n     = W.Button(description="加入購物車", button_style="primary", icon="plus")
    dd_n_size.observe(lambda ch: set_preview(dd_n_size.value), names="value")
    set_preview(dd_n_size.value)
    def on_cb_need(change):
        dd_n_sauce.disabled = not cb_need_sauce.value
        if not cb_need_sauce.value: dd_n_sauce.value = "none"
    cb_need_sauce.observe(on_cb_need, names="value"); on_cb_need(None)
    def add_n(_):
        nsize, sauce = dd_n_size.value, dd_n_sauce.value
        sauce_name = NAME_NUGGETS_SAUCE[sauce] if cb_need_sauce.value else NAME_NUGGETS_SAUCE["none"]
        name = f"麥克雞塊 {NAME_NUGGETS_SIZE[nsize]}（{sauce_name}）"
        add_to_cart(f"{nsize}|{sauce if cb_need_sauce.value else 'none'}", name, PRICES_NUGGETS[nsize], qty_n.value, "snack")
        status_msg.value = f"<span style='color:green'>已加入：{name} × {qty_n.value}</span>"
    btn_add_n.on_click(add_n)
    nuggets_card = W.VBox([W.HBox([dd_n_size, cb_need_sauce, dd_n_sauce]), W.HBox([qty_n, btn_add_n])])

    # 其他點心
    dd_other      = W.Dropdown(options=[(NAME_SNACK_OTHER[k],k) for k in PRICES_SNACK_OTHER], value="apple_pie", description="品項")
    qty_other     = W.IntText(value=1, min=1, step=1, description="份數")
    btn_add_other = W.Button(description="加入購物車", button_style="primary", icon="plus")
    dd_other.observe(lambda ch: set_preview(dd_other.value), names="value")
    set_preview(dd_other.value)
    def add_other(_):
        pid = dd_other.value
        add_to_cart(pid, NAME_SNACK_OTHER[pid], PRICES_SNACK_OTHER[pid], qty_other.value, "snack")
        status_msg.value = f"<span style='color:green'>已加入：{NAME_SNACK_OTHER[pid]} × {qty_other.value}</span>"
    btn_add_other.on_click(add_other)
    other_card = W.VBox([W.HBox([dd_other]), W.HBox([qty_other, btn_add_other])])

    area = W.VBox()
    def rebuild_area(_=None):
        if dd_kind.value=="fries": area.children=[fries_card]; set_preview(dd_fries_size.value)
        elif dd_kind.value=="nuggets": area.children=[nuggets_card]; set_preview(dd_n_size.value)
        else: area.children=[other_card]; set_preview(dd_other.value)
    dd_kind.observe(rebuild_area, names="value"); rebuild_area()
    return W.VBox([dd_kind, area, status_msg])

def build_drink_card():
    dd_item = W.Dropdown(options=[(NAME_DRINK[k],k) for k in PRICES_DRINK], value="coke", description="飲料")
    qty     = W.IntText(value=1, min=1, step=1, description="份數")
    btn_add = W.Button(description="加入購物車", button_style="primary", icon="plus")
    status_msg.value = ""
    dd_item.observe(lambda ch: set_preview(dd_item.value), names="value")
    set_preview(dd_item.value)
    def _add(_):
        pid = dd_item.value
        add_to_cart(pid, NAME_DRINK[pid], PRICES_DRINK[pid], qty.value, "drink")
        status_msg.value = f"<span style='color:green'>已加入：{NAME_DRINK[pid]} × {qty.value}</span>"
    btn_add.on_click(_add)
    return W.VBox([W.HBox([dd_item, qty, btn_add]), status_msg])

def rebuild_product_area(_=None):
    if tb_category.value=="main": product_area.children=[build_main_card()]
    elif tb_category.value=="snack": product_area.children=[build_snack_card()]
    else: product_area.children=[build_drink_card()]
tb_category.observe(rebuild_product_area, names="value"); rebuild_product_area()

# ========= 結帳（按下才產生收據） =========
def generate_receipt(_):
    out_receipt.clear_output()
    order_id = next_order_id()
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    tax_id, carrier, note = txt_tax_id.value.strip(), txt_carrier.value.strip(), txt_note.value.strip()
    tax_line = "（未提供）" if not tax_id else tax_id + ("（統編有效）" if tw_tax_id_ok(tax_id) else "（格式需為 8 碼數字，仍可結帳）")
    carrier_line = "（未提供）" if not carrier else carrier + ("" if carrier_hint_ok(carrier) else "（載具格式與常見型式不符，仍可結帳）")

    lines=[]
    if not CART and not cb_bag.value: lines += ["(提示) 尚未選購任何品項",""]
    lines += [f"訂單編號：{order_id}", f"結帳時間：{ts}", f"統一編號：{tax_line}", f"載具：{carrier_line}", f"備註：{note if note else '（未提供）'}", "", "— 訂單明細 —"]
    for item in CART.values():
        name, unit, qty = item["name_display"], item["unit_price"], item["qty"]
        lines.append(f"- {name} × {qty}   {currency(unit)}" if qty==1 else f"- {name} × {qty}   {currency(unit)} × {qty} = {currency(unit*qty)}")
    if cb_bag.value: lines.append("購物袋：$1")
    subtotal_wo_bag = sum(i["unit_price"]*i["qty"] for i in CART.values())
    total = subtotal_wo_bag + (1 if cb_bag.value else 0)
    lines += ["--------------------------------", f"小計：{currency(subtotal_wo_bag)}", f"總計：{currency(total)}"]
    with out_receipt:
        print("\n".join(lines))

btn_checkout.on_click(generate_receipt)
cb_bag.observe(lambda ch: update_subtotal_label(), names="value")

# 初始渲染（預覽先載入大麥克）
set_preview("bigmac")
render_cart()
display(title, ui)

HTML(value='<h2>🍔 麥當勞點餐機（ipywidgets / PNG）</h2>')

In [8]:
pip install qrcode[pil] pillow ipywidgets

   ---------------------------------------- 0.0/46.0 kB ? eta -:--:--
   -------- ------------------------------- 10.2/46.0 kB ? eta -:--:--
   ----------------- ---------------------- 20.5/46.0 kB 330.3 kB/s eta 0:00:01
   ----------------------------------- ---- 41.0/46.0 kB 326.8 kB/s eta 0:00:01
   ---------------------------------------- 46.0/46.0 kB 325.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [56]:
# --- 麥當勞點餐機 (ipywidgets) 第二代— 左60/右40 + 三分頁 + 飲料冰塊
# + 會員電話/統編/載具 驗證（可留白）
# + 結帳：確認/取消 → 支付方式專屬頁（LinePay/街口/全支付/台灣Pay 顯示QR；櫃台/信用卡顯示提示）
# + 支付專屬頁按「下一步」→ （櫃台/信用卡：只顯示意見回饋；其餘：顯示感謝詞＋回饋）
# + 回饋頁不顯示「返回支付方式」按鈕（僅「送出」）
# + 送出回饋：依支付方式顯示最終提示（帶入尾四碼），並重置回主餐分頁
# 需要套件：pip install qrcode[pil] pillow ipywidgets

from IPython.display import display
import ipywidgets as W
from datetime import datetime
import re, os, io
from PIL import Image
import qrcode

# ========= 版面參數 =========
LEFT_FLEX  = "3 1 0%"    # 左欄約 60%
RIGHT_FLEX = "2 1 0%"    # 右欄約 40%
IMG_BOX_W  = 520
IMG_BOX_H  = 300
FORM_AREA_H = 160        # 分頁表單固定高度（讓按鈕固定在底部）

# ========= 價格/名稱資料 =========
PRICES_MAIN = {"bigmac": 89, "mcchicken": 79, "filetofish": 95, "dblcheese": 99}
PRICES_FRIES = {"fries_s": 35, "fries_m": 45, "fries_l": 55}
PRICES_NUGGETS = {"nuggets4": 49, "nuggets6": 65, "nuggets10": 109}
PRICES_SNACK_OTHER = {"apple_pie": 42, "corn_soup": 55, "side_salad": 50, "sundae_choco": 45}

PRICES_DRINK = {"coke": 38, "sprite": 36, "iced_tea": 32, "hot_coffee": 40}
NAME_DRINK = {"coke":"可口可樂","sprite":"雪碧","iced_tea":"紅茶","hot_coffee":"美式咖啡"}
NAME_MAIN  = {"bigmac":"大麥克","mcchicken":"麥香雞","filetofish":"麥香魚","dblcheese":"雙層牛肉吉事堡"}
NAME_FRIES_SIZE = {"fries_s":"小薯","fries_m":"中薯","fries_l":"大薯"}
NAME_FRIES_SALT = {"nosalt":"無鹽","salt":"有鹽"}
NAME_NUGGETS_SIZE = {"nuggets4":"4塊","nuggets6":"6塊","nuggets10":"10塊"}
NAME_NUGGETS_SAUCE = {"none":"不需要醬料","sweet_chili":"甜辣醬","mustard":"芥末醬","bbq":"BBQ醬"}
NAME_SNACK_OTHER = {"apple_pie":"蘋果派","corn_soup":"玉米濃湯","side_salad":"田園沙拉","sundae_choco":"巧克力聖代"}

# 飲料冰塊
ICE_OPTIONS = {"light":"微冰","less":"少冰","normal":"正常冰","hot":"熱"}

# 支付方式
PAY_METHODS = [
    ("櫃台支付","cashier"),
    ("LinePay","linepay"),
    ("街口支付","jkpay"),
    ("全支付","pxpay"),
    ("台灣Pay","twpay"),
    ("信用卡","credit"),
]
PAY_METHOD_NAMES = {
    "cashier":"櫃台支付","linepay":"LinePay","jkpay":"街口支付","pxpay":"全支付","twpay":"台灣Pay","credit":"信用卡"
}

# 圖片檔名（放在 ./assets/，至少準備 none.png）
IMG_MAP = {
    "bigmac":"bigmac.png","mcchicken":"mcchicken.png","filetofish":"filetofish.png","dblcheese":"dblcheese.png",
    "fries_s":"fries_s.png","fries_m":"fries_m.png","fries_l":"fries_l.png",
    "nuggets4":"nuggets4.png","nuggets6":"nuggets6.png","nuggets10":"nuggets10.png",
    "apple_pie":"apple_pie.png","corn_soup":"corn_soup.png","side_salad":"side_salad.png","sundae_choco":"sundae_choco.png",
    "coke":"coke.png","sprite":"sprite.png","iced_tea":"iced_tea.png","hot_coffee":"hot_coffee.png",
}
ASSETS_DIR = "./assets"
DEFAULT_IMG = "none.png"

ORDER_COUNTER = {"n": 0}
CART = {}                     # sku -> {name_display, unit_price, qty, category}
ORDER_LOCKED = {"v": False}   # 確認後不可更改
LAST_ORDER = {"id": None, "ts": None, "subtotal": 0, "total": 0, "satisfaction": None, "feedback": ""}

# 草稿訂單（尚未送出前固定編號）
DRAFT_ORDER = {"id": None, "ts": None}

# ========= 小工具 =========
def currency(n: int) -> str:
    return f"${n:,}"

def next_order_id() -> str:
    ORDER_COUNTER["n"] += 1
    return f"MCD-{datetime.now().strftime('%Y%m%d-%H%M%S')}-{ORDER_COUNTER['n']:04d}"

def get_tail4_from_oid(oid: str) -> str:
    """取訂單尾數四位（0001~9999 循環）。確保第 10000 筆回到 0001。"""
    m = re.search(r'-(\d+)$', oid or "")
    if not m: return "????"
    n = int(m.group(1))
    tail = ((n - 1) % 9999) + 1
    return f"{tail:04d}"

def tw_tax_id_ok(b: str) -> bool:
    if not b: return True
    return bool(re.fullmatch(r"\d{8}", b))

def carrier_hint_ok(c: str) -> bool:
    if not c: return True
    return bool(re.fullmatch(r"/[A-Z0-9.+-]{7}", c.upper()))

def is_phone_ok(p: str) -> bool:
    if not p: return True
    return bool(re.fullmatch(r"09\d{8}", p))

def _load_png_bytes(path: str, max_w=None, max_h=None) -> bytes:
    if not os.path.exists(path): return b""
    with Image.open(path) as im:
        im = im.convert("RGBA")
        if max_w and max_h:
            ow, oh = im.size
            scale = min(max_w/ow, max_h/oh, 1.0)  # 只縮不放
            if scale < 1.0:
                im = im.resize((int(ow*scale), int(oh*scale)), resample=Image.LANCZOS)
        bio = io.BytesIO(); im.save(bio, format="PNG")
        return bio.getvalue()

def load_image_fixed_box(key: str, box_w=IMG_BOX_W, box_h=IMG_BOX_H):
    filename = IMG_MAP.get(key, DEFAULT_IMG) or DEFAULT_IMG
    path = os.path.join(ASSETS_DIR, filename)
    if not os.path.exists(path): path = os.path.join(ASSETS_DIR, DEFAULT_IMG)
    return _load_png_bytes(path, box_w, box_h)

def make_qr_bytes(payload: str, box=10, border=2, fit_size=(280, 280)) -> bytes:
    qr = qrcode.QRCode(version=None, error_correction=qrcode.constants.ERROR_CORRECT_M, box_size=box, border=border)
    qr.add_data(payload); qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white").convert("RGBA")
    if fit_size:
        ow, oh = img.size
        max_w, max_h = fit_size
        scale = min(max_w/ow, max_h/oh, 1.0)
        if scale < 1.0:
            img = img.resize((int(ow*scale), int(oh*scale)), resample=Image.NEAREST)
    bio = io.BytesIO(); img.save(bio, format="PNG")
    return bio.getvalue()

# ========= 左欄：分頁 + 圖片 + 餐點明細 =========
title_left = W.HTML("<b>餐點選擇（分頁）</b>")

img_preview = W.Image(format="png", layout=W.Layout(max_width="100%", max_height="100%"))
img_box = W.Box([img_preview], layout=W.Layout(
    width=f"{IMG_BOX_W}px", height=f"{IMG_BOX_H}px",
    align_items="center", justify_content="center",
    border="1px solid #ddd", padding="6px"
))

title_detail = W.HTML("<b>餐點明細：</b>")
detail_lines = W.HTML("<i>尚未選取餐點</i>", layout=W.Layout(white_space="pre-wrap"))
detail_panel = W.VBox([title_detail, detail_lines],
                      layout=W.Layout(width="100%", height="140px", overflow_y="auto"))

def set_preview(key: str): img_preview.value = load_image_fixed_box(key)
def set_detail(text: str): detail_lines.value = text

def add_to_cart(sku: str, name_display: str, unit_price: int, qty: int, category: str):
    if ORDER_LOCKED["v"]: return
    if qty <= 0: return
    if sku in CART: CART[sku]["qty"] += qty
    else: CART[sku] = {"name_display": name_display, "unit_price": unit_price, "qty": qty, "category": category}
    render_cart()

# ---- 主餐分頁 ----
def make_tab_main():
    dd_item = W.Dropdown(options=[(NAME_MAIN[k], k) for k in PRICES_MAIN], value="bigmac", description="主餐")
    qty     = W.IntText(value=1, min=1, step=1, description="份數")
    btn_add = W.Button(description="加入購物車", button_style="primary", icon="plus",
                       layout=W.Layout(width="100%"))

    def refresh(_=None):
        pid = dd_item.value
        set_preview(pid)
        set_detail(f"已選：{NAME_MAIN[pid]}；單價 {currency(PRICES_MAIN[pid])}；份數 {qty.value}")

    def on_add(_):
        pid = dd_item.value
        add_to_cart(pid, NAME_MAIN[pid], PRICES_MAIN[pid], qty.value, "main")
        refresh()

    dd_item.observe(refresh, names="value"); qty.observe(refresh, names="value")
    btn_add.on_click(on_add); refresh()

    return W.VBox([W.HBox([dd_item, qty]), W.HTML(""), btn_add],
                  layout=W.Layout(width="100%", min_height=f"{FORM_AREA_H}px", justify_content="space-between"))

# ---- 點心分頁 ----
def make_tab_snack():
    dd_kind = W.ToggleButtons(options=[("薯條","fries"),("雞塊","nuggets"),("其他","other")], value="fries", description="類型")

    # 薯條
    dd_f_size = W.Dropdown(options=[(NAME_FRIES_SIZE[k],k) for k in PRICES_FRIES], value="fries_m", description="尺寸")
    dd_f_salt = W.Dropdown(options=[("無鹽","nosalt"),("有鹽","salt")], value="salt", description="口味")
    qty_f     = W.IntText(value=1, min=1, step=1, description="份數")

    # 麥克雞塊
    dd_n_size = W.Dropdown(options=[(NAME_NUGGETS_SIZE[k],k) for k in PRICES_NUGGETS], value="nuggets6", description="份量")
    cb_sauce  = W.Checkbox(value=True, description="需要醬料")
    dd_sauce  = W.Dropdown(options=[("不需要","none"),("甜辣醬","sweet_chili"),("芥末醬","mustard"),("BBQ醬","bbq")],
                           value="sweet_chili", description="醬料")
    qty_n     = W.IntText(value=1, min=1, step=1, description="份數")

    # 其他點心
    dd_o  = W.Dropdown(options=[(NAME_SNACK_OTHER[k],k) for k in PRICES_SNACK_OTHER], value="apple_pie", description="品項")
    qty_o = W.IntText(value=1, min=1, step=1, description="份數")

    btn_add = W.Button(description="加入購物車", button_style="primary", icon="plus",
                       layout=W.Layout(width="100%"))
    body = W.VBox()

    def f_refresh(_=None):
        set_preview(dd_f_size.value)
        name = f"{NAME_FRIES_SIZE[dd_f_size.value]}（{NAME_FRIES_SALT[dd_f_salt.value]}）"
        set_detail(f"已選：{name}；單價 {currency(PRICES_FRIES[dd_f_size.value])}；份數 {qty_f.value}")

    def n_refresh(_=None):
        set_preview(dd_n_size.value)
        sauce_name = NAME_NUGGETS_SAUCE[dd_sauce.value] if cb_sauce.value else NAME_NUGGETS_SAUCE["none"]
        name = f"麥克雞塊 {NAME_NUGGETS_SIZE[dd_n_size.value]}（{sauce_name}）"
        set_detail(f"已選：{name}；單價 {currency(PRICES_NUGGETS[dd_n_size.value])}；份數 {qty_n.value}")

    def o_refresh(_=None):
        set_preview(dd_o.value)
        set_detail(f"已選：{NAME_SNACK_OTHER[dd_o.value]}；單價 {currency(PRICES_SNACK_OTHER[dd_o.value])}；份數 {qty_o.value}")

    def on_cb(_):
        dd_sauce.disabled = not cb_sauce.value
        if not cb_sauce.value: dd_sauce.value = "none"

    def rebuild(_=None):
        if dd_kind.value=="fries":
            body.children = [W.HBox([dd_f_size, dd_f_salt]), W.HBox([qty_f])]; f_refresh()
        elif dd_kind.value=="nuggets":
            body.children = [W.HBox([dd_n_size, cb_sauce, dd_sauce]), W.HBox([qty_n])]; n_refresh()
        else:
            body.children = [W.HBox([dd_o]), W.HBox([qty_o])]; o_refresh()

    for w in (dd_f_size, dd_f_salt, qty_f): w.observe(f_refresh, names="value")
    for w in (dd_n_size, dd_sauce, qty_n): w.observe(n_refresh, names="value")
    for w in (dd_o, qty_o): w.observe(o_refresh, names="value")
    cb_sauce.observe(on_cb, names="value"); on_cb(None)
    dd_kind.observe(rebuild, names="value"); rebuild()

    def on_add(_):
        if dd_kind.value=="fries":
            name = f"{NAME_FRIES_SIZE[dd_f_size.value]}（{NAME_FRIES_SALT[dd_f_salt.value]}）"
            add_to_cart(f"{dd_f_size.value}|{dd_f_salt.value}", name, PRICES_FRIES[dd_f_size.value], qty_f.value, "snack"); f_refresh()
        elif dd_kind.value=="nuggets":
            sauce = dd_sauce.value if cb_sauce.value else "none"
            name = f"麥克雞塊 {NAME_NUGGETS_SIZE[dd_n_size.value]}（{NAME_NUGGETS_SAUCE[sauce]}）"
            add_to_cart(f"{dd_n_size.value}|{sauce}", name, PRICES_NUGGETS[dd_n_size.value], qty_n.value, "snack"); n_refresh()
        else:
            pid = dd_o.value
            add_to_cart(pid, NAME_SNACK_OTHER[pid], PRICES_SNACK_OTHER[pid], qty_o.value, "snack"); o_refresh()

    btn_add.on_click(on_add)

    return W.VBox([W.HBox([dd_kind]), body, btn_add],
                  layout=W.Layout(width="100%", min_height=f"{FORM_AREA_H}px", justify_content="space-between"))

# ---- 飲料分頁（含冰塊）----
def make_tab_drink():
    dd_item = W.Dropdown(options=[(NAME_DRINK[k],k) for k in PRICES_DRINK], value="coke", description="飲料")
    dd_ice  = W.Dropdown(options=[(v,k) for k,v in ICE_OPTIONS.items()], value="normal", description="冰塊")
    qty     = W.IntText(value=1, min=1, step=1, description="份數")
    btn_add = W.Button(description="加入購物車", button_style="primary", icon="plus",
                       layout=W.Layout(width="100%"))

    def refresh(_=None):
        pid = dd_item.value
        set_preview(pid)
        set_detail(f"已選：{NAME_DRINK[pid]}（{ICE_OPTIONS[dd_ice.value]}）；單價 {currency(PRICES_DRINK[pid])}；份數 {qty.value}")

    def on_add(_):
        pid = dd_item.value
        ice = dd_ice.value
        name = f"{NAME_DRINK[pid]}（{ICE_OPTIONS[ice]}）"
        sku  = f"{pid}|ice:{ice}"
        add_to_cart(sku, name, PRICES_DRINK[pid], qty.value, "drink"); refresh()

    for w in (dd_item, dd_ice, qty): w.observe(refresh, names="value")
    btn_add.on_click(on_add); refresh()

    return W.VBox([W.HBox([dd_item, dd_ice, qty]), W.HTML(""), btn_add],
                  layout=W.Layout(width="100%", min_height=f"{FORM_AREA_H}px", justify_content="space-between"))

# 建立分頁
tab_main, tab_snack, tab_drink = make_tab_main(), make_tab_snack(), make_tab_drink()
tabs = W.Tab(children=[tab_main, tab_snack, tab_drink], layout=W.Layout(width="100%"))
tabs.set_title(0,"主餐"); tabs.set_title(1,"點心"); tabs.set_title(2,"飲料")
tabs.observe(lambda ch: set_preview("bigmac" if ch["new"]==0 else "fries_m" if ch["new"]==1 else "coke")
             if ch["name"]=="selected_index" else None, names="selected_index")

left_col = W.VBox(
    [title_left, tabs, W.HTML("<b>圖片預覽</b>"), img_box, detail_panel],
    layout=W.Layout(border="1px solid #bbb", padding="10px", flex=LEFT_FLEX, min_width="0")
)

# ========= 右欄：購物車 + 小計 + 備註 + 結帳 + 收據/支付 =========
title_cart = W.HTML("<b>購物車明細：</b>")
cart_area = W.VBox(layout=W.Layout(width="100%", min_height="260px", max_height="260px", overflow_y="auto"))
lbl_subtotal = W.HTML("<b>小計：$0</b>")

txt_note   = W.Textarea(placeholder="請輸入備註（飲料少冰、不加洋蔥…）", description="備註",
                        layout=W.Layout(width="100%", height="80px"))
cb_bag     = W.Checkbox(value=False, description="需要購物袋 (+1)")
txt_tax_id = W.Text(value="", placeholder="8 碼數字，如 12345678", description="統編")
txt_carrier= W.Text(value="", placeholder="/AB12345（手機條碼，可留白）", description="載具")

# 會員電話 + 提示 + 即時驗證
txt_phone  = W.Text(value="", placeholder="例：0912345678", description="會員電話",
                    layout=W.Layout(width="100%"))
hint_phone = W.HTML("<i>如需集點，請輸入會員電話</i>")
err_phone  = W.HTML("")      # 即時提示
err_tax    = W.HTML("")      # 統編提示
err_carrier= W.HTML("")      # 載具提示

def update_phone_hint(_=None):
    p = txt_phone.value.strip()
    if not p:
        err_phone.value = ""; txt_phone.layout.border = ""
    elif is_phone_ok(p):
        err_phone.value = "<span style='color:green'>格式正確（09 開頭，共 10 碼）</span>"
        txt_phone.layout.border = "1px solid #3aa655"
    else:
        err_phone.value = "<span style='color:#d33'>格式錯誤：請輸入 09 開頭，共 10 碼數字</span>"
        txt_phone.layout.border = "1px solid #d33"

def update_tax_hint(_=None):
    t = txt_tax_id.value.strip()
    if not t:
        err_tax.value = "<span style='color:#666'>（未提供統編亦可結帳）</span>"
        txt_tax_id.layout.border = ""
    elif tw_tax_id_ok(t):
        err_tax.value = "<span style='color:green'>統編格式正確</span>"
        txt_tax_id.layout.border = "1px solid #3aa655"
    else:
        err_tax.value = "<span style='color:#d33'>統編必須為 8 碼數字</span>"
        txt_tax_id.layout.border = "1px solid #d33"

def update_carrier_hint(_=None):
    c = txt_carrier.value.strip()
    if not c:
        err_carrier.value = "<span style='color:#666'>（未提供載具亦可結帳）</span>"
        txt_carrier.layout.border = ""
    elif carrier_hint_ok(c):
        err_carrier.value = "<span style='color:green'>載具格式正確</span>"
        txt_carrier.layout.border = "1px solid #3aa655"
    else:
        err_carrier.value = "<span style='color:#d33'>載具需為 / 開頭 + 7 碼英數或 . + -（例：/AB12345）</span>"
        txt_carrier.layout.border = "1px solid #d33"

# 即時驗證
txt_phone.observe(update_phone_hint, names="value")
txt_tax_id.observe(update_tax_hint, names="value")
txt_carrier.observe(update_carrier_hint, names="value")

btn_checkout = W.Button(description="結帳", button_style="success", icon="credit-card")
out_receipt  = W.Output(layout=W.Layout(
    border="1px dashed #aaa", padding="8px",
    min_height="280px", max_height="520px",
    overflow_y="auto"
))

right_col = W.VBox(
    [title_cart, cart_area, lbl_subtotal, W.HTML("<hr>"), txt_note,
      W.HTML("<b>結帳</b>"), cb_bag,
      txt_tax_id, err_tax,
      txt_carrier, err_carrier,
      txt_phone, hint_phone, err_phone,
      btn_checkout, out_receipt],
    layout=W.Layout(border="1px solid #bbb", padding="10px", flex=RIGHT_FLEX, min_width="0")
)

ui = W.HBox([left_col, right_col],
             layout=W.Layout(width="100%", justify_content="space-between", align_items="flex-start", gap="12px"))

# ========= 購物車 =========
def compute_subtotal() -> int:
    return sum(i["unit_price"]*i["qty"] for i in CART.values()) + (1 if cb_bag.value else 0)

def update_subtotal_label():
    lbl_subtotal.value = f"<b>小計：{currency(compute_subtotal())}</b>"

def remove_cart_item(sku: str):
    if ORDER_LOCKED["v"]: return
    if sku in CART: del CART[sku]
    render_cart()

def on_qty_changed(change, sku: str):
    if change["name"]!="value": return
    if ORDER_LOCKED["v"]:
        render_cart(); return
    try:
        newq = int(change["new"])
    except:
        newq = 0
    newq = max(0, newq)
    if newq == 0:
        remove_cart_item(sku)
    else:
        CART[sku]["qty"] = newq
        render_cart()

def render_cart():
    rows=[]
    if not CART:
        rows.append(W.HTML("<i>購物車目前是空的</i>"))
    else:
        rows.append(W.HBox([
            W.HTML("<b>品項</b>", layout=W.Layout(width="55%")),
            W.HTML("<b>單價</b>", layout=W.Layout(width="12%")),
            W.HTML("<b>份數</b>", layout=W.Layout(width="16%")),
            W.HTML("<b>小計</b>", layout=W.Layout(width="12%")),
            W.HTML("", layout=W.Layout(width="5%")),
        ]))
        for sku, item in CART.items():
            name, unit, qty = item["name_display"], item["unit_price"], item["qty"]
            line = unit * qty
            name_w = W.HTML(name, layout=W.Layout(width="55%"))
            unit_w = W.Label(currency(unit), layout=W.Layout(width="12%"))
            qty_w  = W.IntText(value=qty, min=0, step=1,
                               layout=W.Layout(width="90%"), disabled=ORDER_LOCKED["v"])
            qty_w.observe(lambda ch, s=sku: on_qty_changed(ch, s), names="value")
            sub_w  = W.Label(currency(line), layout=W.Layout(width="12%"))
            btn_del= W.Button(icon="trash", tooltip="刪除",
                              layout=W.Layout(width="32px"), disabled=ORDER_LOCKED["v"])
            btn_del.on_click(lambda b, s=sku: remove_cart_item(s))
            rows.append(W.HBox([
                name_w, unit_w,
                W.HBox([qty_w], layout=W.Layout(width="16%")),
                sub_w, btn_del
            ]))
    cart_area.children = rows
    update_subtotal_label()

# ========= 支付流程（選擇→方法專屬頁→回饋） =========
def render_payment_section():
    out_receipt.clear_output()
    panel = W.VBox()
    with out_receipt:
        display(panel)

    # 1) 選擇支付方式頁（只有「確認」鍵）
    rb = W.RadioButtons(options=PAY_METHODS, description="支付方式", value="cashier")
    btn_go = W.Button(description="確認", button_style="success", icon="check")

    def to_select_page(_=None):
        panel.children = [
            W.HTML("<b>選擇支付方式</b>"),
            rb,
            btn_go
        ]

    # 2) 支付方式專屬頁（按「下一步」進入回饋）
    btn_next   = W.Button(description="下一步", button_style="success", icon="arrow-right")
    btn_method_back = W.Button(description="返回支付方式", icon="arrow-left")

    def to_method_page(_=None):
        method = rb.value
        controls = W.HBox([btn_next, btn_method_back], layout=W.Layout(gap="8px"))

        if method in ("linepay","jkpay","pxpay","twpay"):
            qr_img = W.Image(format="png")
            payload = f"pay://{method}?order={LAST_ORDER['id']}&amount={LAST_ORDER['total']}&ts={LAST_ORDER['ts']}"
            qr_img.value = make_qr_bytes(payload, fit_size=(280, 280))
            qr_box = W.Box([qr_img], layout=W.Layout(
                border="1px solid #ddd", padding="6px",
                width="300px", height="300px",
                align_items="center", justify_content="center"))
            tip = PAY_METHOD_NAMES[method]
            panel.children = [W.HTML(f"<b>{tip} - 請掃描 QR code 完成付款</b>"), qr_box, controls]

        elif method == "cashier":
            oid = LAST_ORDER.get("id") or DRAFT_ORDER.get("id") or ""
            tail4 = get_tail4_from_oid(oid)
            msg = (f"感謝您的訂購，請至櫃台完成付款後，等待叫號取餐（本次訂單編號：{oid}），"
                   f"您的號碼為訂單編號後四碼({tail4})")
            panel.children = [W.HTML(f"<b>{msg}</b>"), controls]

        elif method == "credit":
            oid = LAST_ORDER.get("id") or DRAFT_ORDER.get("id") or ""
            tail4 = get_tail4_from_oid(oid)
            msg = (f"感謝您的訂購，請使用刷卡機完成付款，至櫃台等待叫號取餐（本次訂單編號：{oid}），"
                   f"您的號碼為訂單編號後四碼({tail4})")
            panel.children = [W.HTML(f"<b>{msg}</b>"), controls]

    # 3) 回饋頁（櫃台/信用卡：不顯示感謝詞；其餘顯示）
    sat = W.RadioButtons(
        options=["非常滿意","滿意","普通","不滿意","非常不滿意"],
        description="滿意度", value=None, layout=W.Layout(width="100%")
    )
    feedback = W.Textarea(placeholder="如果願意，請留下您的使用回饋（選填）",
                          description="回饋", layout=W.Layout(width="100%", height="80px"))
    btn_submit = W.Button(description="送出", button_style="primary", icon="paper-plane")

    def to_feedback_page(_=None):
        method = rb.value
        children = []
        if method not in ("cashier","credit"):
            oid = LAST_ORDER.get("id") or DRAFT_ORDER.get("id") or ""
            tail4 = get_tail4_from_oid(oid)
            children += [
                W.HTML(f"<b>感謝您的訂購，請至櫃台等待叫號取餐（本次訂單編號：{oid}），您的號碼為訂單編號後四碼({tail4})</b>"),
                W.HTML("<hr>")
            ]
        children += [
            W.HTML("<b>請填寫本次服務滿意度</b>"),
            sat, feedback,
            W.HBox([btn_submit], layout=W.Layout(gap="8px"))  # 只保留「送出」
        ]
        panel.children = children

    def on_submit_feedback(_):
        # 儲存回饋
        LAST_ORDER["satisfaction"] = sat.value
        LAST_ORDER["feedback"] = feedback.value.strip()

        # 顯示依支付方式分流的最終提示，帶入尾四碼
        oid = LAST_ORDER.get("id") or DRAFT_ORDER.get("id") or ""
        tail4 = get_tail4_from_oid(oid)
        if rb.value == "cashier":
            final_msg = f"您的編號為 {tail4}，請至櫃台完成結帳，並等候叫號取餐，祝您一切順心"
        else:
            final_msg = f"您的編號為 {tail4}，請至櫃台等候叫號取餐，祝您一切順心"

        panel.children = [W.HTML(f"<b>{final_msg}</b>")]
        _reset_after_submit()

    def _reset_after_submit():
        # 1) 清空購物車並解鎖
        CART.clear(); render_cart(); ORDER_LOCKED["v"] = False
        # 2) 右側欄位恢復/清空
        for w in (txt_note, txt_tax_id, txt_carrier, txt_phone, cb_bag, btn_checkout):
            w.disabled = False
        txt_note.value = ""; txt_tax_id.value = ""; txt_carrier.value = ""; txt_phone.value = ""
        cb_bag.value = False
        update_tax_hint(); update_carrier_hint(); update_phone_hint()
        # 3) 切回主餐、重置左側提示
        tabs.selected_index = 0
        set_preview("bigmac")
        set_detail("<i>尚未選取餐點</i>")
        # 4) 重置草稿訂單
        DRAFT_ORDER["id"] = None
        DRAFT_ORDER["ts"] = None

    # 綁定事件
    btn_go.on_click(to_method_page)
    btn_method_back.on_click(to_select_page)
    btn_next.on_click(to_feedback_page)
    btn_submit.on_click(on_submit_feedback)

    to_select_page()

# ========= 結帳（產生收據 + 確認/取消） =========
def generate_receipt(_):
    if ORDER_LOCKED["v"]:
        render_payment_section()
        return

    # 檢查欄位
    phone   = txt_phone.value.strip()
    tax_id  = txt_tax_id.value.strip()
    carrier = txt_carrier.value.strip()
    update_phone_hint(); update_tax_hint(); update_carrier_hint()

    err_msgs = []
    if not is_phone_ok(phone):
        err_msgs.append("會員電話格式錯誤：請輸入 09 開頭，共 10 碼數字（或留白）。")
    if not tw_tax_id_ok(tax_id):
        err_msgs.append("統編格式錯誤：必須為 8 碼數字（或留白）。")
    if not carrier_hint_ok(carrier):
        err_msgs.append("載具格式錯誤：需 / 開頭 + 7 碼英數或 . + -（或留白）。")

    if err_msgs:
        out_receipt.clear_output()
        with out_receipt:
            display(W.HTML("<span style='color:#d33'><b>請先修正以下欄位：</b><br>" + "<br>".join(err_msgs) + "</span>"))
        return

    # 取得或建立「草稿訂單」編號與時間（固定到送出為止）
    if not DRAFT_ORDER["id"]:
        DRAFT_ORDER["id"] = next_order_id()
        DRAFT_ORDER["ts"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    order_id = DRAFT_ORDER["id"]
    ts       = DRAFT_ORDER["ts"]
    note     = txt_note.value.strip()

    tax_line     = "（未提供）" if not tax_id  else f"{tax_id}（統編有效）"
    carrier_line = "（未提供）" if not carrier else f"{carrier}（載具格式正確）"
    phone_line   = "（未提供）" if not phone   else phone

    # 收據內容
    out_receipt.clear_output()
    lines=[]
    if not CART and not cb_bag.value: lines += ["(提示) 尚未選購任何品項",""]
    lines += [
        f"訂單編號：{order_id}", f"結帳時間：{ts}",
        f"統一編號：{tax_line}", f"載具：{carrier_line}",
        f"會員電話：{phone_line}",
        f"備註：{note if note else '（未提供）'}", "", "— 訂單明細 —"
    ]
    for item in CART.values():
        name, unit, qty = item["name_display"], item["unit_price"], item["qty"]
        if qty == 1:
            lines.append(f"- {name} × {qty}   {currency(unit)}")
        else:
            lines.append(f"- {name} × {qty}   {currency(unit)} × {qty} = {currency(unit*qty)}")
    if cb_bag.value: lines.append("購物袋：$1")
    subtotal_wo_bag = sum(i["unit_price"]*i["qty"] for i in CART.values())
    total = subtotal_wo_bag + (1 if cb_bag.value else 0)
    lines += ["--------------------------------", f"小計：{currency(subtotal_wo_bag)}", f"總計：{currency(total)}", ""]

    # 確認 / 取消
    btn_confirm = W.Button(description="確認送出（鎖定）", button_style="success", icon="check")
    btn_cancel  = W.Button(description="取消（回去修改）", icon="undo")

    def on_confirm(_):
        ORDER_LOCKED["v"] = True
        # 鎖右側輸入
        for w in (txt_note, txt_tax_id, txt_carrier, txt_phone, cb_bag, btn_checkout):
            w.disabled = True
        render_cart()
        # 把草稿訂單定格到 LAST_ORDER，支付頁用它產 QR / 顯示訊息
        LAST_ORDER.update({"id": order_id, "ts": ts, "subtotal": subtotal_wo_bag, "total": total})
        render_payment_section()

    def on_cancel(_):
        ORDER_LOCKED["v"] = False
        with out_receipt:
            print("已取消送出，請在左側或購物車繼續修改後再結帳。")

    with out_receipt:
        print("\n".join(lines))
        display(W.HBox([btn_confirm, btn_cancel], layout=W.Layout(gap="8px")))

    btn_confirm.on_click(on_confirm)
    btn_cancel.on_click(on_cancel)

# 綁定事件
btn_checkout.on_click(generate_receipt)
cb_bag.observe(lambda ch: update_subtotal_label(), names="value")

# ========= 初始化 =========
update_tax_hint(); update_carrier_hint(); update_phone_hint()
set_preview("bigmac")
render_cart()
display(W.HTML("<h3>🍔 麥當勞點餐機（左60/右40・支付方式專屬頁（選擇頁僅確認）・回饋頁僅送出・送出依支付方式顯示最終提示）</h3>"), ui)


HTML(value='<h3>🍔 麥當勞點餐機（左60/右40・支付方式專屬頁（選擇頁僅確認）・回饋頁僅送出・送出依支付方式顯示最終提示）</h3>')